In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import shutil

base_model_path = "/home/gbarbadillo/data/Phi-3-mini-128k-instruct"
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240724_first_trainings/15_continue_training_phi3_4e5/checkpoint-22800'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240724_first_trainings/22_random_question_lr1e-4_1e5dataset/checkpoint-6228'
output_path = "/home/gbarbadillo/data/Phi-3-count"


base_model_path = '/home/gbarbadillo/data/llama-3.1-transformers-8b-instruct-v1'
lora_path = '/mnt/hdd0/Kaggle/arc24/models/20240724_first_trainings/22_llama31_lr1e-4_1e5dataset_r32/checkpoint-6553'
output_path = '/home/gbarbadillo/data/llama-3.1-count'

base_model = AutoModelForCausalLM.from_pretrained(base_model_path, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
if 'llama' in base_model_path:
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    base_model.resize_token_embeddings(len(tokenizer))


model = PeftModel.from_pretrained(base_model, lora_path)
merged_model = model.merge_and_unload()
print('Copying the base model to the output path')
shutil.copytree(base_model_path, output_path)
print('Saving the merged model to the output path')
merged_model.save_pretrained(output_path)